In [1]:
%run ~/CPPN/utils.py

from torch.utils.data import DataLoader
from torch_geometric.utils import negative_sampling

import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, SAGEConv

from ogb.linkproppred import PygLinkPropPredDataset, Evaluator

In [50]:
dataset = PygLinkPropPredDataset(name='ogbl-ddi',
                                 transform=T.ToSparseTensor())
data = dataset[0]
adj_t = data.adj_t.to(device)
adj = adj_t.to_dense()

split_edge = dataset.get_edge_split()

torch.manual_seed(12345)
idx = torch.randperm(split_edge['train']['edge'].size(0))
idx = idx[:split_edge['valid']['edge'].size(0)]
split_edge['eval_train'] = {'edge': split_edge['train']['edge'][idx]}

evaluator = Evaluator(name='ogbl-ddi')

train_neg_edge = negative_sampling(split_edge['train']['edge'].T, num_nodes=adj.shape[0],
                                 num_neg_samples=split_edge['train']['edge'].size(0), method='dense')

split_edge['train']['edge_neg'] = train_neg_edge.T

r = list(split_edge['train']['edge'][:, 0].detach().cpu().numpy()) + list(split_edge['train']['edge_neg'][:, 0].detach().cpu().numpy())
c = list(split_edge['train']['edge'][:, 1].detach().cpu().numpy()) + list(split_edge['train']['edge_neg'][:, 1].detach().cpu().numpy())


In [26]:
class Attention1D(nn.Module):
    def __init__(self, input_dim):
        super(Attention1D, self).__init__()
        self.attention_weights = nn.Parameter(torch.randn(input_dim))
    
    def forward(self, x):
        """
        Args:
        - x: Tensor of shape (batch_size, input_dim)
        
        Returns:
        - output: Tensor of shape (batch_size, input_dim) after applying attention
        """
        # Compute attention scores

        x = x[:, :, :, None]
        attn_scores = torch.matmul(x, self.attention_weights)
        
        # Apply softmax to attention scores
        attn_scores = F.softmax(attn_scores, dim = -2).squeeze(-1)
        
        # Multiply input by attention scores

        output = x.squeeze(-1) * attn_scores
        
        return output

class CPPN(torch.nn.Module):
    
    def __init__(self, in_feats, num_classes = None, num_layers = 2, num_hidden = 128, dropout = 0.1):
        super(CPPN, self).__init__()
        
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.in_feats = in_feats
        self.num_classes = num_classes
        self.dropout = dropout
        
        # self.attention =  Attention1D(input_dim = 1)
        self.lins = torch.nn.ModuleList()
        self.lins.append(torch.nn.Linear(self.in_feats, self.num_hidden))
        # self.class_layer_zero = torch.nn.Linear(self.num_hidden, self.num_hidden)
        # self.class_layer = torch.nn.Linear(self.num_hidden, self.num_classes)
        
        for _ in range(self.num_layers - 2):
            self.lins.append(torch.nn.Linear(self.num_hidden, self.num_hidden))
            
        self.lins.append(torch.nn.Linear(self.num_hidden, 1))
    
    def reset_parameters(self):
        for lin in self.lins:
            lin.reset_parameters()
            
    def forward(self, x):

        # x = self.attention(x)
        
        for idx, lin in enumerate(self.lins[:-1]):
            x = lin(x)
            x = F.dropout(x, p = self.dropout, training = self.training)
            x = F.relu(x)

        # c = self.class_layer_zero(x)
        # c = self.class_layer(c)
        
        x = self.lins[-1](x)
        
        return x #torch.sigmoid(x)

In [5]:
graph = nx.Graph(adj.detach().cpu().numpy())
ig = igraph.Graph([[e[0], e[1]] for e in nx.to_edgelist(graph)])

In [6]:
# len_adj = np.zeros((adj.shape[0], adj.shape[1]))
# for i in tqdm(range((adj.shape[0]))):
#     for j in range(i, (adj.shape[0])):
#         len_adj[i][j] = len_adj[j][i] = nx.shortest_path_length(graph, i, j)

In [7]:
dgi = DGIEmbedding(graph = graph, 
           embed_dim = 64, 
           feature_matrix = None, 
           batch_size = 1, 
           patience = 25,
           model_name = '_')

dgi.embed()
embed = dgi.get_embedding()

/home/shafi.z/CPPN/DGI/utils/process.py:211: UserWarning: torch.sparse.SparseTensor(indices, values, shape, *, device=) is deprecated.  Please use torch.sparse_coo_tensor(indices, values, shape, dtype=, device=). (Triggered internally at /opt/conda/conda-bld/pytorch_1716905979055/work/torch/csrc/utils/tensor_new.cpp:621.)
  return torch.sparse.FloatTensor(indices, values, shape)
 19%|█▉        | 471/2500 [00:13<00:59, 34.01it/s]


In [8]:
del dgi

In [9]:
core_nodes = list(nx.k_core(graph).nodes)[:25] + list(np.random.choice(len(graph), 25))
core_lengths = np.array([[nx.shortest_path_length(graph, i, n) for n in core_nodes] for i in tqdm(range(len(graph)))])
norm_core_lengths = (core_lengths - np.min(core_lengths)) / np.ptp(core_lengths)

a1 = adj @ adj
a2 = a1 @ adj
a3 = a2 @ adj
a4 = a3 @ adj


a1 = a1 - torch.diag(a1) * torch.eye(a1.shape[0], a1.shape[1]).to(device)
a2 = a2 - torch.diag(a2) * torch.eye(a1.shape[0], a1.shape[1]).to(device)
a3 = a3 - torch.diag(a3) * torch.eye(a1.shape[0], a1.shape[1]).to(device)
a4 = a4 - torch.diag(a4) * torch.eye(a1.shape[0], a1.shape[1]).to(device)

a1 = a1.detach().cpu().numpy()
a2 = a2.detach().cpu().numpy()
a3 = a3.detach().cpu().numpy()
a4 = a4.detach().cpu().numpy()


hops = np.concatenate([a1, a2, a3, a4], axis = 0)
hops = (hops - np.min(hops)) / np.ptp(hops)

a1 = hops[:a1.shape[0], :]
a2 = hops[a1.shape[0]:2*a1.shape[0], :]
a3 = hops[2*a1.shape[0]:3*a1.shape[0], :]
a4 = hops[3*a1.shape[0]:4*a1.shape[0], :]



# Coordinates
x_mat = np.tile((np.arange(adj.shape[0]) / adj.shape[0]), (adj.shape[0], 1)).T 
y_mat = np.tile((np.arange(adj.shape[1]) / adj.shape[1]), (adj.shape[1], 1)).T 

# Node Features
np_feat = (embed @ embed.T)
np_feat = (np_feat - np.min(np_feat)) / np.ptp(np_feat)

# Hop Length 
# norm_len_adj = (len_adj - np.min(len_adj)) / np.ptp(len_adj)

# Curvature
deg = torch.sum(adj, axis = 1)
curv = 4 - deg.unsqueeze(1) - deg.unsqueeze(0)
curv = (curv - torch.min(curv)) / (torch.max(curv) - torch.min(curv))
curv = curv.detach().cpu().numpy()

# Input Matrix
in_mat = np.stack([x_mat, y_mat, np_feat, curv, a1, a2, a3, a4], -1)

# Structural Features 
# sense_feat_dict, sense_features = get_sense_features(graph)
# feat_map = np.zeros((sense_features.shape[0], sense_features.shape[0], 2 * sense_features.shape[1]))
# for i in range(sense_features.shape[0]):
#     for j in range(sense_features.shape[0]):
#         feat_map[i, j, :] = np.concatenate([sense_features[i], sense_features[j]])

# Lengths to Core Nodes
core_map = np.zeros((norm_core_lengths.shape[0], norm_core_lengths.shape[0], 2 * norm_core_lengths.shape[1]))
for i in range(norm_core_lengths.shape[0]):
    for j in range(norm_core_lengths.shape[0]):
        core_map[i, j, :] = np.concatenate([norm_core_lengths[i], norm_core_lengths[j]])

in_mat = np.concatenate([in_mat, core_map], axis = -1)

in_mat = torch.Tensor(in_mat)
in_mat = in_mat.to(device)


100%|██████████| 4267/4267 [00:13<00:00, 315.57it/s]


In [99]:
num_feats = 7 + core_map.shape[-1] #+ feat_map.shape[-1]

cppn = CPPN(in_feats = in_mat.shape[-1], #+ context.shape[0],
            num_layers = 2,
            num_classes = 0,
            num_hidden = 7000,
            dropout = 0.5).to(device)

optimizer = optim.Adam(cppn.parameters(),
                       lr = 5e-4,
                       weight_decay = 1e-6)
sum(p.numel() for p in cppn.parameters() if p.requires_grad) 

770002

In [126]:
# t = torch.nn.Transformer(d_model = 1, nhead = 1, dim_feedforward = 2048, num_encoder_layers = 6, num_decoder_layers = 6).to(device)

# optimizer = optim.Adam(t.parameters(),
#                        lr = 5e-4,
#                        weight_decay = 1e-6)

# sum(p.numel() for p in t.parameters() if p.requires_grad) 

# t.train()
# batch_size = 512
# batch_splits = list(np.arange(0, in_mat.shape[0] * in_mat.shape[1], batch_size))
# if in_mat.shape[0] * in_mat.shape[1] not in batch_splits:
#     batch_splits.append(adj.shape[0])

# src = in_mat.reshape(in_mat.shape[-1], -1, 1).to(device)
# tgt = adj.reshape(1, -1, 1).to(device)

# mask = np.zeros(adj.shape)
# mask[r, c] = 1
# mask = torch.Tensor(mask).to(device)
# mask = mask.reshape(1, -1, 1)

# for e in range(1):
#     epoch_loss = []
#     for idx in tqdm(range(len(batch_splits) - 1)):
    
#         optimizer.zero_grad()
#         batch_src = src[:, batch_splits[idx]:batch_splits[idx + 1]]
#         batch_tgt = tgt[:, batch_splits[idx]:batch_splits[idx + 1]]
#         batch_mask = mask[:, batch_splits[idx]:batch_splits[idx + 1]]
        
#         out = t(batch_src, batch_tgt)
#         diff = (out - batch_tgt) * batch_mask
#         batch_loss = torch.sum(torch.square(diff)) 
#         batch_loss.backward()
#         optimizer.step()
#         epoch_loss.append(batch_loss.item())
            
#     print ("Epoch : ", e, "Loss : ", np.sum(epoch_loss), end = '\r')
    

In [100]:
mask = np.zeros(adj.shape)
mask[r, c] = 1
mask = torch.Tensor(mask).to(device)
batch_size = 256
batch_splits = list(np.arange(0, in_mat.shape[0], batch_size))
if adj.shape[0] not in batch_splits:
    batch_splits.append(adj.shape[0])

In [101]:
weight_mask = adj.detach().cpu().numpy().copy()
weight_mask[weight_mask == 0] = 1e-1
weight_mask = torch.Tensor(weight_mask).to(device)

In [102]:
cppn.train()
for e in range(250):

    epoch_loss = []
    for r_idx in range(len(batch_splits) - 1):

        for c_idx in range(len(batch_splits) - 1):
        
            optimizer.zero_grad()
            
            batch_in = in_mat[batch_splits[r_idx]:batch_splits[r_idx + 1],
                              batch_splits[c_idx]:batch_splits[c_idx + 1],
                              :]
            batch_adj = adj[batch_splits[r_idx]:batch_splits[r_idx + 1], batch_splits[c_idx]:batch_splits[c_idx + 1]]
            batch_mask = mask[batch_splits[r_idx]:batch_splits[r_idx + 1], batch_splits[c_idx]:batch_splits[c_idx + 1]]
            batch_weight_mask = weight_mask[batch_splits[r_idx]:batch_splits[r_idx + 1], batch_splits[c_idx]:batch_splits[c_idx + 1]]
    
            batch_out = cppn(batch_in)
            batch_out = batch_out.reshape(batch_in.shape[0], batch_in.shape[1])
            
            diff = (batch_out - batch_adj) * batch_mask * batch_weight_mask
            batch_loss = torch.sum(torch.square(diff)) 
        
            batch_loss.backward()
            optimizer.step()
            epoch_loss.append(batch_loss.item())
        
    print ("Epoch : ", e, "Loss : ", np.sum(epoch_loss), end = '\r')

In [17]:
torch.save(cppn.state_dict(), './cppn_lp.pkl')

In [104]:
# cppn.load_state_dict(torch.load('./cppn_lp.pkl'))

recon_adj = np.zeros(adj.shape)

for r_idx in tqdm(range(len(batch_splits) - 1)):

    for c_idx in range(len(batch_splits) - 1):
        
        batch_in = in_mat[batch_splits[r_idx]:batch_splits[r_idx + 1],
                          batch_splits[c_idx]:batch_splits[c_idx + 1],
                          :]
        batch_out = cppn(batch_in)
        batch_out = batch_out.reshape(batch_in.shape[0], batch_in.shape[1])
    
        recon_adj[batch_splits[r_idx]:batch_splits[r_idx + 1], batch_splits[c_idx]:batch_splits[c_idx + 1]] = batch_out.detach().cpu().numpy()
recon_adj = (recon_adj + recon_adj.T) / 2

  0%|          | 0/17 [00:00<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 1.71 GiB. GPU 

In [ ]:
recon_adj = torch.Tensor(recon_adj).to(device)
deg = torch.sum(adj, axis = 1)
curv = 4 - deg.unsqueeze(1) - deg.unsqueeze(0)

best_diff = np.inf
best_t = None
for t in np.arange(100) / 100: 
    recon_deg = torch.sum(recon_adj > t, axis = 1)
    recon_curv = 4 - recon_deg.unsqueeze(1) - recon_deg.unsqueeze(0)
    c_diff = torch.sum(torch.abs(recon_curv - curv))
    if c_diff < best_diff: 
        best_diff = c_diff
        best_t = t

recon_deg = torch.sum(recon_adj > best_t, axis = 1)
recon_curv = 4 - recon_deg.unsqueeze(1) - recon_deg.unsqueeze(0)

plt.hist(recon_curv.detach().cpu().numpy().flatten(), bins = 100)
plt.hist(curv.detach().cpu().numpy().flatten(), bins = 100)

plt.show()

In [ ]:
plt.imshow(recon_adj.detach().cpu().numpy() > best_t)

In [ ]:
p = [recon_adj[a,b].item() for a, b in split_edge['test']['edge'].detach().cpu().numpy()]
n = [recon_adj[a,b].item() for a, b in split_edge['test']['edge_neg'].detach().cpu().numpy()]
preds = p + n
print ("AUC : ", roc_auc_score([1 for _ in range(len(p))] + [0 for _ in range(len(n))], preds))
results = {}
k = 20
evaluator.K = k
test_hits = evaluator.eval({
            'y_pred_pos': np.array(p),
            'y_pred_neg': np.array(n),
        })[f'hits@' + str(k)]
print ("Hits@" + str(k) + " : ", test_hits)

In [ ]:
optimizer = optim.Adam(cppn.parameters(),
                       lr = 5e-4,
                       weight_decay = 1e-4)

cppn.train()
for e in range(1000):

    epoch_loss = []
    for r_idx in range(len(batch_splits) - 1):

        for c_idx in range(len(batch_splits) - 1):
        
            optimizer.zero_grad()
            
            batch_in = in_mat[batch_splits[r_idx]:batch_splits[r_idx + 1],
                              batch_splits[c_idx]:batch_splits[c_idx + 1],
                              :]
            batch_adj = adj[batch_splits[r_idx]:batch_splits[r_idx + 1], batch_splits[c_idx]:batch_splits[c_idx + 1]]
            batch_mask = mask[batch_splits[r_idx]:batch_splits[r_idx + 1], batch_splits[c_idx]:batch_splits[c_idx + 1]]
    
            batch_out = cppn(batch_in)
            batch_out = batch_out.reshape(batch_in.shape[0], batch_in.shape[1])
            
            diff = (batch_out - batch_adj) * batch_mask
            batch_loss = torch.sum(torch.square(diff)) 
        
            batch_loss.backward()
            optimizer.step()
            epoch_loss.append(batch_loss.item())
        
    print ("Epoch : ", e, "Loss : ", np.sum(epoch_loss), end = '\r')#, "Curvature Loss : ", curv_loss.item(), ")", end = '\r')

In [15]:
np.sum(epoch_loss)

152183.36853790283